In [66]:
import pandas as pd
df = pd.read_pickle("../data/proccessed/retail_clean.pkl")

In [67]:
# df['Lag_1'] = df.groupby('Store')['Weekly_Sales'].shift(1)
# df['Lag_2'] = df.groupby('Store')['Weekly_Sales'].shift(2)
# df['Lag_4'] = df.groupby('Store')['Weekly_Sales'].shift(4)
# df['Lag_52'] = df.groupby('Store')['Weekly_Sales'].shift(52)
# df.head()
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Week,Quarter,dayOfYear,IsMonthStart,IsMonthEnd
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106,2010,2,5,1,36,0,0
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106,2010,2,6,1,43,0,0
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,2010,2,7,1,50,0,0
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,2010,2,8,1,57,0,0
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106,2010,3,9,1,64,0,0


# sure there not missing weeks or strange duplicate  for every Store

#check the sorted of the data for store again to sure that 

In [68]:
df = df.sort_values(['Store', 'Date']).reset_index(drop = True)

#the count of diff between the current raw and the prev raw for every Store

In [69]:
df['prev_date'] = df.groupby('Store')['Date'].shift(1)
df['days_diff'] = (df['Date'] - df['prev_date']).dt.days
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Week,Quarter,dayOfYear,IsMonthStart,IsMonthEnd,prev_date,days_diff
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106,2010,2,5,1,36,0,0,NaT,NaN
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106,2010,2,6,1,43,0,0,2010-02-05,7.0
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,2010,2,7,1,50,0,0,2010-02-12,7.0
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,2010,2,8,1,57,0,0,2010-02-19,7.0
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106,2010,3,9,1,64,0,0,2010-02-26,7.0


#summery for every store

In [70]:
g = df.groupby('Store')['days_diff'].agg(['count','min','median','max']).reset_index()
print(g.head(20))

    Store  count  min  median  max
0       1    142  7.0     7.0  7.0
1       2    142  7.0     7.0  7.0
2       3    142  7.0     7.0  7.0
3       4    142  7.0     7.0  7.0
4       5    142  7.0     7.0  7.0
5       6    142  7.0     7.0  7.0
6       7    142  7.0     7.0  7.0
7       8    142  7.0     7.0  7.0
8       9    142  7.0     7.0  7.0
9      10    142  7.0     7.0  7.0
10     11    142  7.0     7.0  7.0
11     12    142  7.0     7.0  7.0
12     13    142  7.0     7.0  7.0
13     14    142  7.0     7.0  7.0
14     15    142  7.0     7.0  7.0
15     16    142  7.0     7.0  7.0
16     17    142  7.0     7.0  7.0
17     18    142  7.0     7.0  7.0
18     19    142  7.0     7.0  7.0
19     20    142  7.0     7.0  7.0


#show that raw that not have the diff of 7 

In [71]:
bad = df[(df['days_diff'].notna()) & (df['days_diff'] != 7)]
print("None-weekly gaps (should be empty idealy) :", bad.shape[0])
bad.head()

None-weekly gaps (should be empty idealy) : 0


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Week,Quarter,dayOfYear,IsMonthStart,IsMonthEnd,prev_date,days_diff


#known the prev sales for retail because it's the strongest sign to accpected the next sales

In [72]:
lags = [1, 2, 3, 4]
for l in lags:
    df[f'lag_{l}'] = df.groupby('Store')['Weekly_Sales'].shift(l)

#check the first raws with lags

In [73]:
df[['Store', 'Date', 'Weekly_Sales', 'lag_1','lag_2','lag_3','lag_4',]]

,Store,Date,Weekly_Sales,lag_1,lag_2,lag_3,lag_4
0,1,2010-02-05,1643690.90,NaN,NaN,NaN,NaN
1,1,2010-02-12,1641957.44,1643690.90,NaN,NaN,NaN
2,1,2010-02-19,1611968.17,1641957.44,1643690.90,NaN,NaN
3,1,2010-02-26,1409727.59,1611968.17,1641957.44,1643690.90,NaN
4,1,2010-03-05,1554806.68,1409727.59,1611968.17,1641957.44,1643690.90
...,...,...,...,...,...,...,...
6430,45,2012-09-28,713173.95,723086.20,702238.27,766512.66,734297.87
6431,45,2012-10-05,733455.07,713173.95,723086.20,702238.27,766512.66
6432,45,2012-10-12,734464.36,733455.07,713173.95,723086.20,702238.27
6433,45,2012-10-19,718125.53,734464.36,733455.07,713173.95,723086.20


# Rolling Windows Feature 

#roll_mean = Gives Short_Term Sales Trend, for Reduces the impact of one-week volatility.

#roll_median = The Same Idea as roll_mean but it reduces the effect of ouliers.

#roll_std = It Measures the fluctution of sales during the recent period, which is important for understanding volatility.

#roll_mean_8 = Long-term-average gives a broader image about the long-term trend, helps the model to capture seasonal patterns or ongoing changes.

In [74]:
rolling_windows = [4, 8]

for window in rolling_windows:
    df[f'roll_mean_{window}'] = df.groupby('Store')['Weekly_Sales'] \
        .transform(lambda x : x.rolling(window = window, min_periods = 1).mean())
    df[f'roll_median_{window}'] = df.groupby('Store')['Weekly_Sales'] \
        .transform(lambda x : x.rolling(window = window, min_periods = 1).median())
    df[f'roll_std_{window}'] = df.groupby('Store')['Weekly_Sales'] \
        .transform(lambda x : x.rolling(window = window, min_periods = 1).std().fillna(0))
        
df[['Store', 'Date', 'Weekly_Sales', 'roll_mean_4',
    'roll_median_4','roll_std_4','roll_mean_8',
    'roll_median_8','roll_std_8']].head(20)

,Store,Date,Weekly_Sales,roll_mean_4,roll_median_4,roll_std_4,roll_mean_8,roll_median_8,roll_std_8
0,1,2010-02-05,1643690.90,1.643691e+06,1643690.900,0.000000,1.643691e+06,1643690.900,0.000000
1,1,2010-02-12,1641957.44,1.642824e+06,1642824.170,1225.741321,1.642824e+06,1642824.170,1225.741321
2,1,2010-02-19,1611968.17,1.632539e+06,1641957.440,17835.791719,1.632539e+06,1641957.440,17835.791719
3,1,2010-02-26,1409727.59,1.576836e+06,1626962.805,112353.415114,1.576836e+06,1626962.805,112353.415114
4,1,2010-03-05,1554806.68,1.554615e+06,1583387.425,103135.002548,1.572430e+06,1611968.170,97798.393770
5,1,2010-03-12,1439541.59,1.504011e+06,1497174.135,95360.050839,1.550282e+06,1583387.425,102931.283655
6,1,2010-03-19,1472515.79,1.469148e+06,1456028.690,62599.457150,1.539173e+06,1554806.680,98452.947243
7,1,2010-03-26,1404429.92,1.467823e+06,1456028.690,64308.381016,1.522330e+06,1513661.235,102848.040932
8,1,2010-04-02,1594968.28,1.477864e+06,1456028.690,82871.762296,1.516239e+06,1513661.235,95838.501238
9,1,2010-04-09,1545418.53,1.504333e+06,1508967.160,83458.043354,1.504172e+06,1508967.160,82960.025155


# Grouth & Momentum Feature

#pct_change_1 = the percentage change compared to the previous week represents the instantaneous trend

#pct_change_4 = The Change Rate compared to another average 4 weeks (roll_mean_4),represents a short-term trend and reveals unusual altitudes or declines

In [75]:
df['pct_change_1'] = df.groupby('Store')['Weekly_Sales'].pct_change()
df['pct_change_1'] = df['pct_change_1'].fillna(0)  # to handle the nan in the first raws we add in it 0 rather than nan

df['pct_change_4'] = df['Weekly_Sales'] / df['roll_mean_4'] -1

print(df[df['Store']==1].head(20)[['Date','Weekly_Sales', 'pct_change_1', 'pct_change_4']])

         Date  Weekly_Sales  pct_change_1  pct_change_4
0  2010-02-05    1643690.90      0.000000      0.000000
1  2010-02-12    1641957.44     -0.001055     -0.000528
2  2010-02-19    1611968.17     -0.018264     -0.012600
3  2010-02-26    1409727.59     -0.125462     -0.105977
4  2010-03-05    1554806.68      0.102913      0.000123
5  2010-03-12    1439541.59     -0.074135     -0.042865
6  2010-03-19    1472515.79      0.022906      0.002292
7  2010-03-26    1404429.92     -0.046238     -0.043189
8  2010-04-02    1594968.28      0.135670      0.079239
9  2010-04-09    1545418.53     -0.031066      0.027311
10 2010-04-16    1466058.28     -0.051352     -0.024396
11 2010-04-23    1391256.12     -0.051023     -0.072140
12 2010-04-30    1425100.71      0.024327     -0.021866
13 2010-05-07    1603955.12      0.125503      0.089945
14 2010-05-14    1494251.50     -0.068396      0.010557
15 2010-05-21    1399662.07     -0.063302     -0.054757
16 2010-05-28    1432069.95      0.023154     -0

In [76]:
pd.options.display.max_columns = 200
pd.options.display.max_rows = 200
df.head(30)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Week,Quarter,dayOfYear,IsMonthStart,IsMonthEnd,prev_date,days_diff,lag_1,lag_2,lag_3,lag_4,roll_mean_4,roll_median_4,roll_std_4,roll_mean_8,roll_median_8,roll_std_8,pct_change_1,pct_change_4
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106,2010,2,5,1,36,0,0,NaT,NaN,NaN,NaN,NaN,NaN,1.643691e+06,1643690.900,0.000000,1.643691e+06,1643690.900,0.000000,0.000000,0.000000
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106,2010,2,6,1,43,0,0,2010-02-05,7.0,1643690.90,NaN,NaN,NaN,1.642824e+06,1642824.170,1225.741321,1.642824e+06,1642824.170,1225.741321,-0.001055,-0.000528
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,2010,2,7,1,50,0,0,2010-02-12,7.0,1641957.44,1643690.90,NaN,NaN,1.632539e+06,1641957.440,17835.791719,1.632539e+06,1641957.440,17835.791719,-0.018264,-0.012600
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,2010,2,8,1,57,0,0,2010-02-19,7.0,1611968.17,1641957.44,1643690.90,NaN,1.576836e+06,1626962.805,112353.415114,1.576836e+06,1626962.805,112353.415114,-0.125462,-0.105977
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106,2010,3,9,1,64,0,0,2010-02-26,7.0,1409727.59,1611968.17,1641957.44,1643690.90,1.554615e+06,1583387.425,103135.002548,1.572430e+06,1611968.170,97798.393770,0.102913,0.000123
5,1,2010-03-12,1439541.59,0,57.79,2.667,211.380643,8.106,2010,3,10,1,71,0,0,2010-03-05,7.0,1554806.68,1409727.59,1611968.17,1641957.44,1.504011e+06,1497174.135,95360.050839,1.550282e+06,1583387.425,102931.283655,-0.074135,-0.042865
6,1,2010-03-19,1472515.79,0,54.58,2.720,211.215635,8.106,2010,3,11,1,78,0,0,2010-03-12,7.0,1439541.59,1554806.68,1409727.59,1611968.17,1.469148e+06,1456028.690,62599.457150,1.539173e+06,1554806.680,98452.947243,0.022906,0.002292
7,1,2010-03-26,1404429.92,0,51.45,2.732,211.018042,8.106,2010,3,12,1,85,0,0,2010-03-19,7.0,1472515.79,1439541.59,1554806.68,1409727.59,1.467823e+06,1456028.690,64308.381016,1.522330e+06,1513661.235,102848.040932,-0.046238,-0.043189
8,1,2010-04-02,1594968.28,0,62.27,2.719,210.820450,7.808,2010,4,13,2,92,0,0,2010-03-26,7.0,1404429.92,1472515.79,1439541.59,1554806.68,1.477864e+06,1456028.690,82871.762296,1.516239e+06,1513661.235,95838.501238,0.135670,0.079239
9,1,2010-04-09,1545418.53,0,65.86,2.770,210.622857,7.808,2010,4,14,2,99,0,0,2010-04-02,7.0,1594968.28,1404429.92,1472515.79,1439541.59,1.504333e+06,1508967.160,83458.043354,1.504172e+06,1508967.160,82960.025155,-0.031066,0.027311


# Important Week

In [77]:
from src.features import identify_important_weeks

df = identify_important_weeks(df)

In [78]:
df[['Store', 'Date', 'Weekly_Sales','Holiday_Event_Flag', 'High_Sales_Event_Flag', 'Year_End_Flag', 'Important_Week_Flag','Important_Week_Reason']].head(30)

,Store,Date,Weekly_Sales,Holiday_Event_Flag,High_Sales_Event_Flag,Year_End_Flag,Important_Week_Flag,Important_Week_Reason
0,1,2010-02-05,1643690.90,False,0,0,0,Normal
1,1,2010-02-12,1641957.44,True,0,0,1,Holiday
2,1,2010-02-19,1611968.17,False,0,0,0,Normal
3,1,2010-02-26,1409727.59,False,0,0,0,Normal
4,1,2010-03-05,1554806.68,False,0,0,0,Normal
5,1,2010-03-12,1439541.59,False,0,0,0,Normal
6,1,2010-03-19,1472515.79,False,0,0,0,Normal
7,1,2010-03-26,1404429.92,False,0,0,0,Normal
8,1,2010-04-02,1594968.28,False,0,0,0,Normal
9,1,2010-04-09,1545418.53,False,0,0,0,Normal


# Store-level aggregate feature 

#give the model statics information about every store throughout history like Typical sales volume and volatility

In [79]:
store_stats = df.groupby('Store')['Weekly_Sales'].agg(
    store_mean = 'mean',
    store_median = 'median',
    store_std = 'std',
    store_max = 'max',
    store_min = 'min',
).reset_index()

#merge the store_state with the original dataframe 

In [80]:
df = df.merge(store_stats, on = 'Store', how = 'left')

#added the new feature percentage of the current week sales  to store_mean

In [81]:
df['sales_to_store_mean'] = df['Weekly_Sales'] / df['store_mean']

#show some rows to ensure from the results

In [82]:
df[['Store', 'Date', 'Weekly_Sales','store_mean', 'store_std', 'sales_to_store_mean']].head(10)

,Store,Date,Weekly_Sales,store_mean,store_std,sales_to_store_mean
0,1,2010-02-05,1643690.90,1.555264e+06,155980.767761,1.056856
1,1,2010-02-12,1641957.44,1.555264e+06,155980.767761,1.055742
2,1,2010-02-19,1611968.17,1.555264e+06,155980.767761,1.036459
3,1,2010-02-26,1409727.59,1.555264e+06,155980.767761,0.906423
4,1,2010-03-05,1554806.68,1.555264e+06,155980.767761,0.999706
5,1,2010-03-12,1439541.59,1.555264e+06,155980.767761,0.925593
6,1,2010-03-19,1472515.79,1.555264e+06,155980.767761,0.946795
7,1,2010-03-26,1404429.92,1.555264e+06,155980.767761,0.903017
8,1,2010-04-02,1594968.28,1.555264e+06,155980.767761,1.025529
9,1,2010-04-09,1545418.53,1.555264e+06,155980.767761,0.993669


# Interaction Features

In [83]:
df['Holiday_x_store_mean'] = df['Holiday_Event_Flag'] * df['store_mean']
df['Temp_x_week'] = df['Temperature'] * df['Week'] if 'Temperature' in df.columns else 0
df['SalesRatio_x_Holiday'] = df['sales_to_store_mean'] * df['Holiday_Event_Flag']

df[['Store', 'Date', 'Holiday_Event_Flag', 'store_mean', 'Holiday_x_store_mean', 'SalesRatio_x_Holiday']].head(10)


,Store,Date,Holiday_Event_Flag,store_mean,Holiday_x_store_mean,SalesRatio_x_Holiday
0,1,2010-02-05,False,1.555264e+06,0.000000e+00,0.000000
1,1,2010-02-12,True,1.555264e+06,1.555264e+06,1.055742
2,1,2010-02-19,False,1.555264e+06,0.000000e+00,0.000000
3,1,2010-02-26,False,1.555264e+06,0.000000e+00,0.000000
4,1,2010-03-05,False,1.555264e+06,0.000000e+00,0.000000
5,1,2010-03-12,False,1.555264e+06,0.000000e+00,0.000000
6,1,2010-03-19,False,1.555264e+06,0.000000e+00,0.000000
7,1,2010-03-26,False,1.555264e+06,0.000000e+00,0.000000
8,1,2010-04-02,False,1.555264e+06,0.000000e+00,0.000000
9,1,2010-04-09,False,1.555264e+06,0.000000e+00,0.000000


# Encoding & Imputation

#late Feature Encoding or Feature Finalization & Model Preparation  the goal from it to prepare the data to be ready 100% to enter to the model and the opetation use in it to prepare it is that 1-أزالة الفارغ
2-تحويل المتغيرات لفورم يفهمه الموديل
3-تقليل التعقيد خصوصا المتجر
4-تعتبر اخر محطة قبل النمذجة

In [84]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

# ----- 1) نسخة من البيانات -----
df_encoded = df.copy()

# ===========================
#     IMPUTATION (Numeric)
# ===========================
numeric_cols = df_encoded.select_dtypes(include=['float64', 'int64']).columns

num_imputer = SimpleImputer(strategy='median')
df_encoded[numeric_cols] = num_imputer.fit_transform(df_encoded[numeric_cols])

# ==================================
#     TARGET ENCODING for Store
# ==================================

# نحسب متوسط المبيعات لكل متجر
store_mean_sales = df_encoded.groupby("Store")["Weekly_Sales"].mean().to_dict()

# نستبدل رقم المتجر بمتوسط المبيعات الخاص به
df_encoded["Store_Encoded"] = df_encoded["Store"].map(store_mean_sales)

# يمكن حذف العمود الأصلي أو الاحتفاظ به
# df_encoded.drop("Store", axis=1, inplace=True)

# ===========================
#  (Optional) Categorical Imputation
# ===========================
cat_cols = df_encoded.select_dtypes(include=['object', 'category']).columns

if len(cat_cols) > 0:
    cat_imputer = SimpleImputer(strategy='most_frequent')
    df_encoded[cat_cols] = cat_imputer.fit_transform(df_encoded[cat_cols])

df_encoded.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Week,Quarter,dayOfYear,IsMonthStart,IsMonthEnd,prev_date,days_diff,lag_1,lag_2,lag_3,lag_4,roll_mean_4,roll_median_4,roll_std_4,roll_mean_8,roll_median_8,roll_std_8,pct_change_1,pct_change_4,Holiday_Event_Flag,High_Sales_Event_Flag,Year_End_Flag,Important_Week_Flag,Important_Week_Reason,store_mean,store_median,store_std,store_max,store_min,sales_to_store_mean,Holiday_x_store_mean,Temp_x_week,SalesRatio_x_Holiday,Store_Encoded
0,1.0,2010-02-05,1643690.90,0.0,42.31,2.572,211.096358,8.106,2010,2,5,1,36,0.0,0.0,NaT,7.0,960971.975,960998.52,961041.30,960476.1,1.643691e+06,1643690.900,0.000000,1.643691e+06,1643690.900,0.000000,0.000000,0.000000,False,0.0,0.0,0.0,Normal,1.555264e+06,1534849.64,155980.767761,2387950.2,1316899.31,1.056856,0.000000e+00,211.55,0.000000,1.555264e+06
1,1.0,2010-02-12,1641957.44,1.0,38.51,2.548,211.242170,8.106,2010,2,6,1,43,0.0,0.0,2010-02-05,7.0,1643690.900,960998.52,961041.30,960476.1,1.642824e+06,1642824.170,1225.741321,1.642824e+06,1642824.170,1225.741321,-0.001055,-0.000528,True,0.0,0.0,1.0,Holiday,1.555264e+06,1534849.64,155980.767761,2387950.2,1316899.31,1.055742,1.555264e+06,231.06,1.055742,1.555264e+06
2,1.0,2010-02-19,1611968.17,0.0,39.93,2.514,211.289143,8.106,2010,2,7,1,50,0.0,0.0,2010-02-12,7.0,1641957.440,1643690.90,961041.30,960476.1,1.632539e+06,1641957.440,17835.791719,1.632539e+06,1641957.440,17835.791719,-0.018264,-0.012600,False,0.0,0.0,0.0,Normal,1.555264e+06,1534849.64,155980.767761,2387950.2,1316899.31,1.036459,0.000000e+00,279.51,0.000000,1.555264e+06
3,1.0,2010-02-26,1409727.59,0.0,46.63,2.561,211.319643,8.106,2010,2,8,1,57,0.0,0.0,2010-02-19,7.0,1611968.170,1641957.44,1643690.90,960476.1,1.576836e+06,1626962.805,112353.415114,1.576836e+06,1626962.805,112353.415114,-0.125462,-0.105977,False,0.0,0.0,0.0,Normal,1.555264e+06,1534849.64,155980.767761,2387950.2,1316899.31,0.906423,0.000000e+00,373.04,0.000000,1.555264e+06
4,1.0,2010-03-05,1554806.68,0.0,46.50,2.625,211.350143,8.106,2010,3,9,1,64,0.0,0.0,2010-02-26,7.0,1409727.590,1611968.17,1641957.44,1643690.9,1.554615e+06,1583387.425,103135.002548,1.572430e+06,1611968.170,97798.393770,0.102913,0.000123,False,0.0,0.0,0.0,Normal,1.555264e+06,1534849.64,155980.767761,2387950.2,1316899.31,0.999706,0.000000e+00,418.50,0.000000,1.555264e+06


#here drop the store and save it in that copy in eda so we remove it here to make model not confuse with that because it confuse it

In [85]:
df_encoded.drop('Store', axis = 1, inplace = True)


In [86]:
df_encoded

,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Week,Quarter,dayOfYear,IsMonthStart,IsMonthEnd,prev_date,days_diff,lag_1,lag_2,lag_3,lag_4,roll_mean_4,roll_median_4,roll_std_4,roll_mean_8,roll_median_8,roll_std_8,pct_change_1,pct_change_4,Holiday_Event_Flag,High_Sales_Event_Flag,Year_End_Flag,Important_Week_Flag,Important_Week_Reason,store_mean,store_median,store_std,store_max,store_min,sales_to_store_mean,Holiday_x_store_mean,Temp_x_week,SalesRatio_x_Holiday,Store_Encoded
0,2010-02-05,1643690.90,0.0,42.31,2.572,211.096358,8.106,2010,2,5,1,36,0.0,0.0,NaT,7.0,960971.975,960998.52,961041.30,960476.10,1.643691e+06,1643690.900,0.000000,1.643691e+06,1643690.900,0.000000,0.000000,0.000000,False,0.0,0.0,0.0,Normal,1.555264e+06,1534849.64,155980.767761,2387950.20,1316899.31,1.056856,0.000000e+00,211.55,0.000000,1.555264e+06
1,2010-02-12,1641957.44,1.0,38.51,2.548,211.242170,8.106,2010,2,6,1,43,0.0,0.0,2010-02-05,7.0,1643690.900,960998.52,961041.30,960476.10,1.642824e+06,1642824.170,1225.741321,1.642824e+06,1642824.170,1225.741321,-0.001055,-0.000528,True,0.0,0.0,1.0,Holiday,1.555264e+06,1534849.64,155980.767761,2387950.20,1316899.31,1.055742,1.555264e+06,231.06,1.055742,1.555264e+06
2,2010-02-19,1611968.17,0.0,39.93,2.514,211.289143,8.106,2010,2,7,1,50,0.0,0.0,2010-02-12,7.0,1641957.440,1643690.90,961041.30,960476.10,1.632539e+06,1641957.440,17835.791719,1.632539e+06,1641957.440,17835.791719,-0.018264,-0.012600,False,0.0,0.0,0.0,Normal,1.555264e+06,1534849.64,155980.767761,2387950.20,1316899.31,1.036459,0.000000e+00,279.51,0.000000,1.555264e+06
3,2010-02-26,1409727.59,0.0,46.63,2.561,211.319643,8.106,2010,2,8,1,57,0.0,0.0,2010-02-19,7.0,1611968.170,1641957.44,1643690.90,960476.10,1.576836e+06,1626962.805,112353.415114,1.576836e+06,1626962.805,112353.415114,-0.125462,-0.105977,False,0.0,0.0,0.0,Normal,1.555264e+06,1534849.64,155980.767761,2387950.20,1316899.31,0.906423,0.000000e+00,373.04,0.000000,1.555264e+06
4,2010-03-05,1554806.68,0.0,46.50,2.625,211.350143,8.106,2010,3,9,1,64,0.0,0.0,2010-02-26,7.0,1409727.590,1611968.17,1641957.44,1643690.90,1.554615e+06,1583387.425,103135.002548,1.572430e+06,1611968.170,97798.393770,0.102913,0.000123,False,0.0,0.0,0.0,Normal,1.555264e+06,1534849.64,155980.767761,2387950.20,1316899.31,0.999706,0.000000e+00,418.50,0.000000,1.555264e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6430,2012-09-28,713173.95,0.0,64.88,3.997,192.013558,8.684,2012,9,39,3,272,0.0,0.0,2012-09-21,7.0,723086.200,702238.27,766512.66,734297.87,7.262528e+05,718130.075,28158.111035,7.266344e+05,722791.565,19148.520975,-0.013708,-0.018009,False,0.0,0.0,0.0,Normal,7.859814e+05,764014.06,130168.526635,1682862.03,617207.58,0.907367,0.000000e+00,2530.32,0.000000,7.859814e+05
6431,2012-10-05,733455.07,0.0,64.89,3.985,192.170412,8.667,2012,10,40,4,279,0.0,0.0,2012-09-28,7.0,713173.950,723086.20,702238.27,766512.66,7.179884e+05,718130.075,13372.247001,7.266867e+05,722791.565,19169.034933,0.028438,0.021542,False,0.0,0.0,0.0,Normal,7.859814e+05,764014.06,130168.526635,1682862.03,617207.58,0.933171,0.000000e+00,2595.60,0.000000,7.859814e+05
6432,2012-10-12,734464.36,0.0,54.47,4.000,192.327265,8.667,2012,10,41,4,286,0.0,0.0,2012-10-05,7.0,733455.070,713173.95,723086.20,702238.27,7.260449e+05,728270.635,10003.552464,7.281826e+05,728270.635,19262.101459,0.001376,0.011596,False,0.0,0.0,0.0,Normal,7.859814e+05,764014.06,130168.526635,1682862.03,617207.58,0.934455,0.000000e+00,2233.27,0.000000,7.859814e+05
6433,2012-10-19,718125.53,0.0,56.47,3.969,192.330854,8.667,2012,10,42,4,293,0.0,0.0,2012-10-12,7.0,734464.360,733455.07,713173.95,723086.20,7.248047e+05,725790.300,10770.694653,7.281692e+05,728270.635,19270.013089,-0.022246,-0.009215,False,0.0,0.0,0.0,Normal,7.859814e+05,764014.06,130168.526635,1682862.03,617207.58,0.913667,0.000000e+00,2371.74,0.000000,7.859814e+05


In [87]:
df_encoded.to_pickle("../data/proccessed/data_encoded.pkl")